# My Experiment - PIMA Indians Diabetes Prediction with Deep Learning

While the previous notebook in the folder was about hyperparamter tuning by grid-search for the prediction of PIMA Indians Diabetes, this notebook tries to focus on the weaknesses, which were observed in the previous notebook. Fact is:
* There was no train test split, so it was not easy to judge how good the model was, when it will be applied on any unseen data.
* The dataset was imbalanced, leading to the result that the prediction for the minority class (1) was less good than the majority class (0).

In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
seed = 6
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
fileName = "diabetes.csv"
columns = ['n_pregnant', 'glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(fileName, names = columns, header=0)

In [4]:
df

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
columns = ['glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.dropna(inplace=True)
df

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1
...,...,...,...,...,...,...,...,...,...
753,0,181.0,88.0,44.0,510.0,43.3,0.222,26,1
755,1,128.0,88.0,39.0,110.0,36.5,1.057,37,1
760,2,88.0,58.0,26.0,16.0,28.4,0.766,22,0
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0


In [8]:
dataset = df.values
print(dataset.shape)

X = dataset[:,0:8]
y = dataset[:, 8].astype(int)

print(X.shape)
print(y.shape)

(392, 9)
(392, 8)
(392,)


The previous steps are pretty similar compared to the first notebook. We dropped the NaN values, 393 rows are remaining. Having `X` and `y`: We will split the data into train and test sets, which means the amount of training material will be less than 392, but we try to not cause underfitting.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=True)

print(X_train.shape)
print(X_test.shape)

(313, 8)
(79, 8)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
print(scaler)

X_standardized = scaler.transform(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [11]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

The previous notebook showed which parameters were working well in each of the experimented combinations. I will apply some findings in this notebook.

In [12]:
def create_model():
    model = Sequential()
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='linear'))
    model.add(Dense(2, input_dim = 4, kernel_initializer='normal', activation='linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________
None


Again, as the goal of this week is experiment with grid-search, we will continue the experiment trying to find the best batch size and number of epochs. The difference is, this time I added the paramter class weight to the fit function. Class 1 is the minority class, that is why I assigned a greater weight to it than to the majority class 0.

In [13]:
# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(shuffle=True, random_state=seed), refit=True, verbose=20, return_train_score=True)
grid_results = grid.fit(X_standardized, y_train, class_weight={0: 1.0, 1: 2.0})

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
250/250 [==============================] - 0s 1ms/step - loss: 0.8385 - accuracy: 0.5760
Epoch 2/10
250/250 [==============================] - 0s 182us/step - loss: 0.6650 - accuracy: 0.7400
Epoch 3/10
250/250 [==============================] - 0s 162us/step - loss: 0.6494 - accuracy: 0.7760
Epoch 4/10
250/250 [==============================] - 0s 171us/step - loss: 0.6331 - accuracy: 0.7840
Epoch 5/10
250/250 [==============================] - 0s 289us/step - loss: 0.6396 - accuracy: 0.7760
Epoch 6/10
250/250 [==============================] - 0s 952us/step - loss: 0.6353 - accuracy: 0.7680
Epoch 7/10
250/250 [==============================] - 0s 516us/step - loss: 0.6329 - accuracy: 0.7880
Epoch 8/10
250/250 [==============================] - 0s 707us/step - loss: 0.6329 - accuracy: 0.7880
Epoch 9/10
250/250 [==============================] - 0s 513us/step - loss: 0.6478 - accuracy: 0.7560
Epoch 10/10
250/250 [==============================] - 0s 105us/step
[CV]  batch_siz

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


Epoch 1/10
250/250 [==============================] - 0s 746us/step - loss: 0.8524 - accuracy: 0.6960
Epoch 2/10
250/250 [==============================] - 0s 176us/step - loss: 0.6711 - accuracy: 0.7560
Epoch 3/10
250/250 [==============================] - 0s 153us/step - loss: 0.6484 - accuracy: 0.7880
Epoch 4/10
250/250 [==============================] - 0s 164us/step - loss: 0.6386 - accuracy: 0.7960
Epoch 5/10
250/250 [==============================] - 0s 148us/step - loss: 0.6356 - accuracy: 0.7960
Epoch 6/10
250/250 [==============================] - 0s 205us/step - loss: 0.6317 - accuracy: 0.8040
Epoch 7/10
250/250 [==============================] - 0s 149us/step - loss: 0.6287 - accuracy: 0.8040
Epoch 8/10
250/250 [==============================] - 0s 175us/step - loss: 0.6319 - accuracy: 0.8080
Epoch 9/10
250/250 [==============================] - 0s 154us/step - loss: 0.6277 - accuracy: 0.8080
Epoch 10/10
250/250 [==============================] - 0s 71us/step
[CV]  batch_si

...

251/251 [==============================] - 0s 62us/step - loss: 0.6632 - accuracy: 0.7769
Epoch 17/100
251/251 [==============================] - 0s 70us/step - loss: 0.6640 - accuracy: 0.7729
Epoch 18/100
251/251 [==============================] - 0s 57us/step - loss: 0.6668 - accuracy: 0.7689
Epoch 19/100
251/251 [==============================] - 0s 66us/step - loss: 0.6668 - accuracy: 0.7689
Epoch 20/100
251/251 [==============================] - 0s 65us/step - loss: 0.6652 - accuracy: 0.7649
Epoch 21/100
251/251 [==============================] - 0s 66us/step - loss: 0.6626 - accuracy: 0.7729
Epoch 22/100
251/251 [==============================] - 0s 73us/step - loss: 0.6621 - accuracy: 0.7809
Epoch 23/100
251/251 [==============================] - 0s 56us/step - loss: 0.6629 - accuracy: 0.7809
Epoch 24/100
251/251 [==============================] - 0s 79us/step - loss: 0.6629 - accuracy: 0.7729
Epoch 25/100
251/251 [==============================] - 0s 89us/step - loss: 0.6633 - 

251/251 [==============================] - 0s 68us/step - loss: 0.6629 - accuracy: 0.7689
Epoch 96/100
251/251 [==============================] - 0s 90us/step - loss: 0.6617 - accuracy: 0.7689
Epoch 97/100
251/251 [==============================] - 0s 98us/step - loss: 0.6650 - accuracy: 0.7729
Epoch 98/100
251/251 [==============================] - 0s 85us/step - loss: 0.6632 - accuracy: 0.7729
Epoch 99/100
251/251 [==============================] - 0s 77us/step - loss: 0.6635 - accuracy: 0.7729
Epoch 100/100
251/251 [==============================] - 0s 54us/step
[CV]  batch_size=40, epochs=100, score=(train=0.773, test=0.806), total=   3.0s
[CV] batch_size=40, epochs=100 .......................................
Epoch 1/100
251/251 [==============================] - 0s 683us/step - loss: 0.8987 - accuracy: 0.6614
Epoch 2/100
251/251 [==============================] - 0s 81us/step - loss: 0.8493 - accuracy: 0.7689
Epoch 3/100
251/251 [==============================] - 0s 63us/step - lo

251/251 [==============================] - 0s 57us/step - loss: 0.6296 - accuracy: 0.7729
Epoch 73/100
251/251 [==============================] - 0s 61us/step - loss: 0.6322 - accuracy: 0.7729
Epoch 74/100
251/251 [==============================] - 0s 54us/step - loss: 0.6298 - accuracy: 0.7769
Epoch 75/100
251/251 [==============================] - 0s 51us/step - loss: 0.6276 - accuracy: 0.7729
Epoch 76/100
251/251 [==============================] - 0s 60us/step - loss: 0.6272 - accuracy: 0.7769
Epoch 77/100
251/251 [==============================] - 0s 57us/step - loss: 0.6281 - accuracy: 0.7729
Epoch 78/100
251/251 [==============================] - 0s 57us/step - loss: 0.6299 - accuracy: 0.7649
Epoch 79/100
251/251 [==============================] - 0s 62us/step - loss: 0.6290 - accuracy: 0.7689
Epoch 80/100
251/251 [==============================] - 0s 65us/step - loss: 0.6285 - accuracy: 0.7769
Epoch 81/100
251/251 [==============================] - 0s 66us/step - loss: 0.6273 - 

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.8min finished


Epoch 1/50
313/313 [==============================] - 0s 639us/step - loss: 0.8853 - accuracy: 0.7220
Epoch 2/50
313/313 [==============================] - 0s 63us/step - loss: 0.7756 - accuracy: 0.7316
Epoch 3/50
313/313 [==============================] - 0s 60us/step - loss: 0.6947 - accuracy: 0.7380
Epoch 4/50
313/313 [==============================] - 0s 51us/step - loss: 0.6826 - accuracy: 0.7508
Epoch 5/50
313/313 [==============================] - 0s 46us/step - loss: 0.6655 - accuracy: 0.7604
Epoch 6/50
313/313 [==============================] - 0s 46us/step - loss: 0.6574 - accuracy: 0.7636
Epoch 7/50
313/313 [==============================] - 0s 46us/step - loss: 0.6530 - accuracy: 0.7700
Epoch 8/50
313/313 [==============================] - 0s 46us/step - loss: 0.6494 - accuracy: 0.7796
Epoch 9/50
313/313 [==============================] - 0s 45us/step - loss: 0.6482 - accuracy: 0.7796
Epoch 10/50
313/313 [==============================] - 0s 48us/step - loss: 0.6459 - accur

In [14]:
def summarize_result(grid_results):
    print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
    print('\n')

    means_train = grid_results.cv_results_['mean_train_score']
    means_test = grid_results.cv_results_['mean_test_score']
    stds = grid_results.cv_results_['std_test_score']
    params = grid_results.cv_results_['params']

    for mean_train, mean_test, stdev, param in zip(means_train, means_test, stds, params):
        print('With: {0} -> mean training accuracy: {1}, mean & std of test accuracy: ({2}, {3})'.format(param, mean_train, mean_test, stdev))

summarize_result(grid_results)

Best: 0.7766001105308533, using {'batch_size': 40, 'epochs': 50}


With: {'batch_size': 10, 'epochs': 10} -> mean training accuracy: 0.7668079614639283, mean & std of test accuracy: (0.7541218757629394, 0.02844921519240059)
With: {'batch_size': 10, 'epochs': 50} -> mean training accuracy: 0.7779601693153382, mean & std of test accuracy: (0.767025101184845, 0.04881930392506942)
With: {'batch_size': 10, 'epochs': 100} -> mean training accuracy: 0.7763792753219605, mean & std of test accuracy: (0.7701997041702271, 0.047857557526636826)
With: {'batch_size': 20, 'epochs': 10} -> mean training accuracy: 0.7819665431976318, mean & std of test accuracy: (0.7669226884841919, 0.03754620986062043)
With: {'batch_size': 20, 'epochs': 50} -> mean training accuracy: 0.7851601719856263, mean & std of test accuracy: (0.767025101184845, 0.04881930392506942)
With: {'batch_size': 20, 'epochs': 100} -> mean training accuracy: 0.7827537775039672, mean & std of test accuracy: (0.770148491859436, 0.0413081858

In [15]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = grid.predict(X_standardized)
print(accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))

X_test_standardized = scaler.transform(X_test)
y_pred = grid.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

313/313 [==============================] - 0s 49us/step
0.7667731629392971
              precision    recall  f1-score   support

           0       0.86      0.79      0.82       213
           1       0.62      0.72      0.66       100

    accuracy                           0.77       313
   macro avg       0.74      0.75      0.74       313
weighted avg       0.78      0.77      0.77       313

79/79 [==============================] - 0s 36us/step
0.8354430379746836
              precision    recall  f1-score   support

           0       0.88      0.86      0.87        49
           1       0.77      0.80      0.79        30

    accuracy                           0.84        79
   macro avg       0.82      0.83      0.83        79
weighted avg       0.84      0.84      0.84        79



Now we have results for the training and test accuracy scores. The test accuracy is even higher than the training accuracy, but we should not overvalue it, as the amount of test data is small. But the interesting observation is, the precision, recall and F1 scores for class 1 have increased. Again, I would not overvalue it, as the test set is small. But it is an interesting observation.